In [1]:
# NOTE: you CAN change this cell
# If you want to use your own database, download it here
!gdown --fuzzy https://drive.google.com/file/d/1RSvUuK9r4UcbpXL7AlMUDRIzisYJh4Rk/view?usp=sharing -O list_province.txt
!gdown --fuzzy https://drive.google.com/file/d/1RSQ7xgC106qeAKNqb1xI1BWEy-g3-V93/view?usp=sharing -O list_district.txt
!gdown --fuzzy https://drive.google.com/file/d/1RV1roTLQ77FwxbtyqbD6zTuB5s_LJMkQ/view?usp=sharing -O list_ward.txt

Downloading...
From: https://drive.google.com/uc?id=1RSvUuK9r4UcbpXL7AlMUDRIzisYJh4Rk
To: /content/list_province.txt
100% 750/750 [00:00<00:00, 2.47MB/s]
Downloading...
From: https://drive.google.com/uc?id=1RSQ7xgC106qeAKNqb1xI1BWEy-g3-V93
To: /content/list_district.txt
100% 7.75k/7.75k [00:00<00:00, 22.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1RV1roTLQ77FwxbtyqbD6zTuB5s_LJMkQ
To: /content/list_ward.txt
100% 88.3k/88.3k [00:00<00:00, 65.8MB/s]


In [2]:
# NOTE: you CAN change this cell
# Add more to your needs
# you must place ALL pip install here
!pip install editdistance

In [3]:
# NOTE: you CAN change this cell
# import your library here
import re
import time

In [4]:
# NOTE: you MUST change this cell
# New methods / functions must be written under class Solution.
class Solution:
    def __init__(self):
        # list provice, district, ward for private test, do not change for any reason
        self.province_path = 'list_province.txt'
        self.district_path = 'list_district.txt'
        self.ward_path = 'list_ward.txt'

        # Tries for correct names
        self.province_trie = self.TrieNode("*")
        self.district_trie = self.TrieNode("*")
        self.ward_trie = self.TrieNode("*")

        # Tries for typo names
        self.province_trie_typo = self.TrieNode("*")
        self.district_trie_typo = self.TrieNode("*")
        self.ward_trie_typo = self.TrieNode("*")

        # Insert data to correct tries
        self.province_trie.add_file(self.province_path)
        self.district_trie.add_file(self.district_path)
        self.ward_trie.add_file(self.ward_path)

        # Insert data to typo tries
        self.province_trie_typo.add_file(self.province_path, typo=True)
        self.district_trie_typo.add_file(self.district_path, typo=True)
        self.ward_trie_typo.add_file(self.ward_path, typo=True)

        # Hardcode some special cases
        self.province_trie.add("ionh", "Hà Nội")
        self.province_trie.add("mch", "Hồ Chí Minh")
        self.province_trie.add("hnimc", "Hồ Chí Minh")
        self.province_trie.add("htt", "Thừa Thiên Huế")
        self.province_trie.add("euhtauht", "Thừa Thiên Huế")
        self.district_trie.add("tb", "Bình Thạnh")
        self.district_trie.add("bt", "Tân Bình")

    def process(self, s: str):
        MAX_NAME_LENGTH = 12 # The longest name is "bariavungtau"

        # Preprocess input string
        # Example:
        # Số 259/54/8, Tổ 28, KP1, Long Bình Tân, Biên Hòa, Đồng Nai.
        # --> sotokp1longbinhtânbienhoađongnai
        # --> iangnođaohneibnâthnibgnol1pkotos
        input = self.preprocess(s)

        # Initialize output dictionary
        output = {"province": "", "district": "", "ward": ""}
        # Tries of correct names
        tries = {"province": self.province_trie, "district": self.district_trie, "ward": self.ward_trie}
        # Tries of typo names
        typo_tries = {"province": self.province_trie_typo, "district": self.district_trie_typo, "ward": self.ward_trie_typo}

        typo_flag = False # True - search in tries of typos, False - search in tries of correct names.
        trie_flag = 0 # 0 - search in trie of provinces, 1 - search in trie of districts, 2 - search in trie of wards.
        current_node = self.province_trie # Initalize current node by root of province trie.
        milestone = -1 # Update by current index if a valid name is found.

        # Loop until valid names of all field are found.
        while True:
            last_terminate_node = current_node # Last found valid name.
            for i in range(milestone+1, milestone+11):
                if i >= len(input):
                    break
                typo_num = 0 # Number of typos in 1 searching process
                last_match_index = i # Last match character index.
                terminate_node_found = False # True - found a valid name, False - otherwise.
                for j in range(i, len(input)):
                    # Check if current string is too long.
                    if j-i>MAX_NAME_LENGTH:
                        break

                    # Search through all the children of the present `node`.
                    child_not_found = True
                    for child in current_node.children:
                        if child.char == input[j]:
                            # Found child that match the character.
                            child_not_found = False
                            # Update last match index.
                            last_match_index = j
                            # Assign node as the child containing the char and break.
                            current_node = child
                            break

                    # If current index is not match in the trie, it is a typo.
                    if j != last_match_index:
                        typo_num += 1
                        # If there are 2 typos in 1 searching process, then out of the loop.
                        if typo_num > 1 or j-last_match_index>2:
                            break

                    # If a child node is found and it is terminate, then update milestone variables.
                    if not child_not_found and current_node.is_terminate:
                        last_terminate_node = current_node
                        milestone = j
                        terminate_node_found = True
                        if not current_node.children:
                            break

                # If a valid name is found, then break and paste it in the appropriate field.
                if terminate_node_found:
                    break
                else:
                    # If not, go back to the last found node and search for other fields.
                    current_node = last_terminate_node

            # Update appropriate field by last found name.
            output_keys = ["province", "district", "ward"]
            output[output_keys[trie_flag]] = last_terminate_node.word if last_terminate_node.is_terminate else output[output_keys[trie_flag]]

            # If no valid name is found in the current correct trie, then search in the corresponding trie of typos.
            if output[output_keys[trie_flag]]=="" and typo_flag==False:
                current_node = typo_tries[output_keys[trie_flag]]
                typo_flag = True
            else:
                # Go to next field tries.
                trie_flag += 1
                # If all tries are searched, out of while loop.
                if trie_flag == 3:
                    break
                current_node = tries[output_keys[trie_flag]]
                typo_flag = False

        # Return output
        return output

    @staticmethod
    def preprocess(s: str, isinput=True):
        def no_accent_vietnamese(s):
            s = re.sub('[áãăắằẳẵặấầẫậà]', 'a', s) # ạâảẩ
            s = re.sub('[ÁÀẢÃẠĂẮẰẲẴẶÂẤẦẨẪẬ]', 'A', s)
            # s = re.sub(u'Đ', 'D', s)
            # s = re.sub(u'đ', 'd', s)
            s = re.sub('[éèẻẽẹêếềểễệ]', 'e', s)
            s = re.sub('[ÉÈẺẼẸÊẾỀỂỄỆ]', 'E', s)
            s = re.sub('[õọóòôốồổỗộơớờởỡợ]', 'o', s) # ỏ
            s = re.sub('[ỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢ]', 'O', s) # ÓÒ
            s = re.sub('[íìỉĩị]', 'i', s)
            s = re.sub('[ÍÌỈĨỊ]', 'I', s)
            s = re.sub('[úủũưứừửữự]', 'u', s) # ù
            s = re.sub('[ÚÙỦŨỤƯỨỪỬỮỰ]', 'U', s)
            s = re.sub('[ýỳỷỹỵ]', 'y', s)
            s = re.sub('[ÝỲỶỸỴ]', 'Y', s)
            return s

        processed_string = s[:-1] if s[-1] == "." else s
        delimiters = []
        if isinput:
            # Define a regex pattern to match numbers not preceded by the word
            # "Quận", "Q", "Q.", "Phường", ...
            pattern = r"(?<!Quận )(?<!Q)(?<!Q\.)(?<!Quận \d)(?<!Q\d)(?<!Q\.\d)(?<!Phường )(?<!P)(?<!P\.)(?<!Phường \d)(?<!P\d)(?<!P\.\d)\d+"
            # Replace matched numbers with an empty string
            processed_string = re.sub(pattern, "", processed_string)

            # Hardcode
            processed_string = processed_string.replace("H.", "")
            processed_string = processed_string.replace("T.Phố", "")

            delimiters = [
                "t.x.",
                "thành phố", "thành fhố", "tỉnh", "tp.",
                "quận", "huyện", "huyên", "thị xã", "q.",  "tx",
                "phường", "xã", "thị trấn", "f.", "x.",
                "khu phố", "khóm", "thôn", "ấp",
                " ", "-", ".", ",", "/",
            ]
            # "t.", "t.t.", "h.",
        else:
            delimiters = ["-", " "]

        processed_string = processed_string.lower()
        for d in delimiters:
            processed_string = processed_string.replace(d, "")
        processed_string = no_accent_vietnamese(processed_string)

        return processed_string[::-1]


    class TrieNode(object):
        def __init__(self, char: str):
            self.char = char
            self.children = []
            # Is it the last character of the word.
            self.is_terminate = False
            self.word = ""

        def add(self, word: str, node_name: str):
            """
            Adding a word in the trie structure
            """
            node = self
            for char in word:
                found_in_child = False
                # Search for the character through all children of the present `node`
                for child in node.children:
                    if child.char == char:
                        # Point the node to the child that contains this char
                        node = child
                        found_in_child = True
                        break
                # We did not find it in the children nodes -> add a new chlid
                if not found_in_child:
                    new_node = self.__class__(char)
                    node.children.append(new_node)
                    # Point node to the new child
                    node = new_node
            # Mark it as the end of a word.
            if node.is_terminate == False or len(node.word)>len(node_name):
                node.is_terminate = True
                node.word = node_name

        def add_item(self, input: str, typo):
            """
            Adding word and its variation in the trie structure
            """
            word = Solution.preprocess(input, isinput=False)
            if typo:
                if len(word) > 5:
                    for i in range(0,len(word)):
                        self.add(word[:i] + word[i+1:], input)
            else:
                self.add(word, input)

        def add_file(self, file_name, typo=False):
            items = []
            with open(file_name, encoding="utf8") as file:
                items = [line.rstrip() for line in file]

            for item in items:
                if not item in ["","I", "V", "Hồ"]:
                    self.add_item(item, typo)

In [12]:
print(Solution().preprocess("59/12 Ng-B-Khiêm, Đa Kao Quận 1, TP Hồ Chí Minh")[::-1])

ngbkhiemđakao1tphochiminh


In [32]:
# NOTE: DO NOT change this cell
# This cell is for downloading private test
!rm -rf test.json
!gdown --fuzzy https://drive.google.com/file/d/1PBt3U9I3EH885CDhcXspebyKI5Vw6uLB/view?usp=sharing -O test.json

Downloading...
From: https://drive.google.com/uc?id=1PBt3U9I3EH885CDhcXspebyKI5Vw6uLB
To: /content/test.json
100% 79.4k/79.4k [00:00<00:00, 51.8MB/s]


In [33]:
# NOTE: DO NOT change this cell
# This cell is for scoring

TEAM_NAME = 'Group2'  # This should be your team name
EXCEL_FILE = f'{TEAM_NAME}.xlsx'

import json
import time
with open('test.json') as f:
    data = json.load(f)

summary_only = True
df = []
solution = Solution()
timer = []
correct = 0
for test_idx, data_point in enumerate(data):
    address = data_point["text"]

    ok = 0
    try:
        start = time.perf_counter_ns()
        result = solution.process(address)
        answer = data_point["result"]
        finish = time.perf_counter_ns()
        timer.append(finish - start)
        ok += int(answer["province"] == result["province"])
        ok += int(answer["district"] == result["district"])
        ok += int(answer["ward"] == result["ward"])
        df.append([
            test_idx,
            address,
            answer["province"],
            result["province"],
            int(answer["province"] == result["province"]),
            answer["district"],
            result["district"],
            int(answer["district"] == result["district"]),
            answer["ward"],
            result["ward"],
            int(answer["ward"] == result["ward"]),
            ok,
            timer[-1] / 1_000_000_000,
        ])
    except Exception as e:
        df.append([
            test_idx,
            address,
            answer["province"],
            "EXCEPTION",
            0,
            answer["district"],
            "EXCEPTION",
            0,
            answer["ward"],
            "EXCEPTION",
            0,
            0,
            0,
        ])
        # any failure count as a zero correct
        pass
    correct += ok


    if not summary_only:
        # responsive stuff
        print(f"Test {test_idx:5d}/{len(data):5d}")
        print(f"Correct: {ok}/3")
        print(f"Time Executed: {timer[-1] / 1_000_000_000:.4f}")


print(f"-"*30)
total = len(data) * 3
score_scale_10 = round(correct / total * 10, 2)
if len(timer) == 0:
    timer = [0]
max_time_sec = round(max(timer) / 1_000_000_000, 4)
avg_time_sec = round((sum(timer) / len(timer)) / 1_000_000_000, 4)

import pandas as pd

df2 = pd.DataFrame(
    [[correct, total, score_scale_10, max_time_sec, avg_time_sec]],
    columns=['correct', 'total', 'score / 10', 'max_time_sec', 'avg_time_sec',],
)

columns = [
    'ID',
    'text',
    'province',
    'province_student',
    'province_correct',
    'district',
    'district_student',
    'district_correct',
    'ward',
    'ward_student',
    'ward_correct',
    'total_correct',
    'time_sec',
]

df = pd.DataFrame(df)
df.columns = columns

print(f'{TEAM_NAME = }')
print(f'{EXCEL_FILE = }')
print(df2)

!pip install xlsxwriter
writer = pd.ExcelWriter(EXCEL_FILE, engine='xlsxwriter')
df2.to_excel(writer, index=False, sheet_name='summary')
df.to_excel(writer, index=False, sheet_name='details')
writer.close()


------------------------------
TEAM_NAME = 'Group2'
EXCEL_FILE = 'Group2.xlsx'
   correct  total  score / 10  max_time_sec  avg_time_sec
0     1216   1350        9.01        0.0903        0.0004
